$$
\DeclareMathOperator\tr{tr}
\DeclareMathOperator\Tr{Tr}
\DeclareMathOperator\dev{dev}
\DeclareMathOperator\Dev{Dev}
\DeclareMathOperator\sph{sph}
\DeclareMathOperator\Sph{Sph}
\newcommand{\bF}{\mathbf{F}}
\newcommand{\bC}{\mathbf{C}}
\newcommand{\bd}{\mathbf{d}}
\newcommand{\bc}{\mathbf{c}}
\newcommand{\bA}{\mathbf{A}}
\newcommand{\bS}{\mathbf{S}}
\newcommand{\Bsigma}{{\boldsymbol{\sigma}}}
\newcommand{\pdiff}[2]{\frac{\partial#1}{\partial#2}}
\nonumber
$$

# General Setup

In [1]:
%matplotlib nbagg

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [3]:
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.comp import IntegrationRuleSpace
from netgen.csg import *
from ngsolve.solvers import Newton

importing NGSolve-6.2.2204


In [4]:
SetNumThreads(1)

cube = OrthoBrick(Pnt(0,0,0), Pnt(1,1,1))
geo = CSGeometry()
geo.Add(cube)

mesh = Mesh(geo.GenerateMesh(maxh=0.2, quad_dominated=False))
#Draw(mesh)

names = [ "bottom", "back", "right", "front", "top", "left", "inner" ]
for i, name in enumerate(names):
    mesh.ngmesh.SetBCName(i, name)
print (mesh.GetBoundaries())

d = mesh.dim

('bottom', 'back', 'right', 'front', 'top', 'left')


In [5]:
import numpy as np

# Parameters

In [6]:
parameters = dict(
    G=1,
    Gprime_f=1e2,
)

In [7]:
G = Parameter(parameters["G"])
Gprime_f = Parameter(parameters["Gprime_f"])
Gprime = Gprime_f * G

# Energy functions

In [8]:
I = Id(d)

In [9]:
def C(F):
    return F.trans * F


def I_1(C):
    return Trace(C)


def I_3(C):
    return Det(C)


# "Deviatoric" part of energy density
def Psi_dev(C, G):
    return G/2 * (I_1(C) - d - log(I_3(C)))


# Purely volumetric part of energy density
def Psi_vol(C, G_prime):
    return G_prime/2 * (sqrt(I_3(C)) - 1)**2


def Psi(C, G, G_prime):
    return Psi_dev(C, G) + Psi_vol(C, G_prime)

In [10]:
def Compile(cf, **options): 
    _realcompile = options.get("realcompile", False)
    return cf.Compile(realcompile=_realcompile)

# A homogeneous BVP

In [11]:
fes_u = VectorH1(mesh, order=2, dirichletx="front", dirichlety="left", dirichletz="top|bottom")

In [12]:
u, u_test = fes_u.TnT()

In [13]:
def F(u):
    return I + Grad(u)

gfu = GridFunction(fes_u)

# Forms

In [14]:
energy_density = Psi(C(F(u)), G, Gprime)

In [15]:
a = BilinearForm(fes_u, symmetric=True)
a += Variation(Compile(energy_density) * dx)

In [16]:
t = 0
gfu.Set((0, 0, 0))
delta_t = Parameter(0.1)

# Run

In [17]:
u_top = Parameter(0)
tol = 1e-10
t = 0
gfu.Set((0, 0, 0))
dfu = GridFunction(fes_u)

In [18]:
def disp(t):
    return t

In [19]:
while t < 1:
    
    with TaskManager(pajetrace=10**8):
        
        print()
        print("t: ", t)
        print("disp(t): ", disp(t))
        dfu.Set((dfu[0], dfu[1], disp(t)), definedon=mesh.Boundaries("top"))
        ret = Newton(a, gfu, inverse="pardiso", dirichletvalues=dfu.vec)
        print(ret)
        t += delta_t.Get()


t:  0
disp(t):  0
Newton iteration  0
err =  0.0
(0, 1)

t:  0.1
disp(t):  0.1
Newton iteration  0
err =  0.0716342945377445
Newton iteration  1
err =  0.00038065647432666366
Newton iteration  2
err =  1.06610570027293e-08
Newton iteration  3
err =  1.6096227459998463e-15
(0, 4)

t:  0.2
disp(t):  0.2
Newton iteration  0
err =  0.05951989234753596
Newton iteration  1
err =  0.00026224108176861686
Newton iteration  2
err =  5.05623934429478e-09
Newton iteration  3
err =  1.6416428905035183e-15
(0, 4)

t:  0.30000000000000004
disp(t):  0.30000000000000004
Newton iteration  0
err =  0.05023814893936566
Newton iteration  1
err =  0.00018651705527436048
Newton iteration  2
err =  2.5562411903842e-09
Newton iteration  3
err =  1.7392921691045712e-15
(0, 4)

t:  0.4
disp(t):  0.4
Newton iteration  0
err =  0.04296999721787357
Newton iteration  1
err =  0.0001362683479428678
Newton iteration  2
err =  1.363735257514115e-09
Newton iteration  3
err =  1.78847506294057e-15
(0, 4)

t:  0.5
disp(t

In [20]:
Draw(gfu[2], mesh)

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2204', 'mesh_dim': 3, 'order2d': 2, 'order3d':…

BaseWebGuiScene